# Setup

In [1]:
%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
#
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Install chromium and chromium-driver
apt-get update
apt-get install chromium chromium-driver

# Install selenium
pip install selenium

Executing: /tmp/apt-key-gpghome.g2Rt13dqJz/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: public key "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.9p2qZgUxrN/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: public key "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.myDNhNz8BQ/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gpg: key 4DFAB270CAA96DFA: public key "Debian Security Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Get:1 http://deb.debian.org/debian buster InRelease [122 kB]
Get:2 http://deb.debian.org/debian bust

Import Libs

In [2]:
import pandas as pd
import os
import requests
import time
import random

from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Crawling

In [3]:
WEBDRIVER_DELAY_TIME_INT = 10

service = Service(executable_path=r'/usr/bin/chromedriver')
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')    #do not show pop up
chrome_options.add_argument('--no-sandbox')
chrome_options.headless = True
driver = webdriver.Chrome(service=service, options=chrome_options)
driver.implicitly_wait(5)                   #prevent timeout error
wait = WebDriverWait(driver, WEBDRIVER_DELAY_TIME_INT)

In [4]:
datasets = []
deletion_script = 'arguments[0].parentNode.removeChild(arguments[0]);'
for page_idx in tqdm(range(1, 11)):
    main_url = f'https://www.thivien.net/searchpoem.php?PoemType=16&ViewType=1&Country=2&Age[]=3&Page={page_idx}'
    driver.get(main_url)

    content_tags_xpath = '//*[@class="page-content container"]/div[2]/div/div[@class="list-item"]'
    content_tags = driver.find_elements(By.XPATH, content_tags_xpath)
    for idx in range(len(content_tags)):
        content_tag_xpath = f'/html/body/div[4]/div[2]/div/div[{2+idx}]'
        content_title_xpath = f'/html/body/div[4]/div[2]/div/div[{2+idx}]/h4/a'
        content_tag = wait.until(EC.presence_of_element_located((By.XPATH, content_tag_xpath)))
        poem_title = wait.until(EC.presence_of_element_located((By.XPATH, content_title_xpath))).text
        poem_url = wait.until(EC.presence_of_element_located((By.XPATH, content_title_xpath))).get_attribute('href')

        try:
            driver.get(poem_url)

            poem_src_xpath = '//div[@class="small"]'
            poem_content_tag = wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'poem-content')))

            try:
                poem_content_i_tag = poem_content_tag.find_element(By.TAG_NAME, 'i')
                driver.execute_script(deletion_script, poem_content_i_tag)
            except:
                pass

            try:
                poem_content_b_tag = poem_content_tag.find_element(By.TAG_NAME, 'b')
                driver.execute_script(deletion_script, poem_content_b_tag)
            except:
                pass

            poem_content = poem_content_tag.text

            try:
                poem_src_tag = wait.until(EC.presence_of_element_located((By.XPATH, poem_src_xpath)))
                poem_src = poem_src_tag.text
            except:
                poem_src = ''

            poem_info = {
                'title': poem_title,
                'content': poem_content,
                'source': poem_src,
                'url': poem_url
            }

            datasets.append(poem_info)

            driver.back()
        except Exception as e:
            print(e)
            print(poem_url)

 20%|██        | 2/10 [00:12<00:47,  5.91s/it]

Message: 
Stacktrace:
#0 0x561078f0b7f9 <unknown>
#1 0x561078eab3b3 <unknown>
#2 0x561078bf3016 <unknown>
#3 0x561078c2781e <unknown>
#4 0x561078c5d8fb <unknown>
#5 0x561078c4aded <unknown>
#6 0x561078c5b9e1 <unknown>
#7 0x561078c4ac93 <unknown>
#8 0x561078c1cce4 <unknown>
#9 0x561078c1e4d2 <unknown>
#10 0x561078ed7542 <unknown>
#11 0x561078ee6ce7 <unknown>
#12 0x561078ee69e4 <unknown>
#13 0x561078eeb13a <unknown>
#14 0x561078ee75b9 <unknown>
#15 0x561078ecce00 <unknown>
#16 0x561078efe5d2 <unknown>
#17 0x561078efe778 <unknown>
#18 0x561078f16a1f <unknown>
#19 0x7b8fd1c3fac3 <unknown>
#20 0x7b8fd1cd1850 <unknown>

https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3%A2n/N%E1%BA%BFu-mu%E1%BB%91n-l%C3%A0m-g%C3%AC-%C4%91%E1%BA%A5y/poem-tTo-G66NSaSIuRYBJ8H9iA


 20%|██        | 2/10 [01:31<06:06, 45.80s/it]


TimeoutException: Message: 
Stacktrace:
#0 0x561078f0b7f9 <unknown>
#1 0x561078eab3b3 <unknown>
#2 0x561078bf3016 <unknown>
#3 0x561078c2781e <unknown>
#4 0x561078c5d8fb <unknown>
#5 0x561078c4aded <unknown>
#6 0x561078c5b9e1 <unknown>
#7 0x561078c4ac93 <unknown>
#8 0x561078c1cce4 <unknown>
#9 0x561078c1e4d2 <unknown>
#10 0x561078ed7542 <unknown>
#11 0x561078ee6ce7 <unknown>
#12 0x561078ee69e4 <unknown>
#13 0x561078eeb13a <unknown>
#14 0x561078ee75b9 <unknown>
#15 0x561078ecce00 <unknown>
#16 0x561078efe5d2 <unknown>
#17 0x561078efe778 <unknown>
#18 0x561078f16a1f <unknown>
#19 0x7b8fd1c3fac3 <unknown>
#20 0x7b8fd1cd1850 <unknown>


In [ ]:
datasets

# Saving

In [ ]:
df = pd.DataFrame(datasets)
df.to_csv('poem_dataset.csv', index=True)

In [ ]:
df

In [ ]:
!zip -r poem_dataset.zip poem_dataset.csv